### Import des librairies nécessaires
- On importe la librairie <code>math</code> de python pour pouvoir utiliser la méthode <code>sqrt</code> dans notre calcul de la division euclidienne
- La librairie random va nous aider à diviser le dataset en deux sets train et test qu'on va voir plus en détails 

In [10]:
from math import sqrt
import random

### Charger les données d'un jeu de données
- On a définit une fonction <code>load_dataset</code> qui prend en paramètre un fichier de type "txt"(par exemple "data.txt"). Ensuite on ouvre le fichier pour le lire ligne par ligne avec la méthode <code>readlines()</code> et on créer une liste <code>data_string</code> qui va être constituée des données du fichier.

In [3]:
def load_dataset(filename):
    with open(filename) as file:
        lines = file.readlines()
        data_string = []
        for line in lines:
            instance_string = line.split("\n")
            """Comme on sépare les données avec le caractère de retour à la ligne "\n", une liste vide [''] est créée, 
            on la supprime donc avec la méthode remove(element)"""
            instance_string.remove('')
            for value_string in instance_string:
                data_string.append(value_string.split(";"))
    return data_string
        
data_string = load_dataset("data.txt")
#data_string_pre_test = load_dataset("preTest.txt")
data_string_final_test = load_dataset("finalTest.txt")

### Convertir les données qui sont de type string en type float
- Après avoir charger le jeu de données dans une liste <code>data_string</code>, nous allons convertir les données de type string en données de type float avec la méthode <code>convert_string_float</code> qui prend en paramètre <code>data_string</code>.

In [4]:
def convert_string_float(data_string):
    data_float = []
    for value in data_string:
        instance_float = []
        """On n'itère pas jusqu'à la fin de la liste car on veut consérver la dernière valeur(la classe) 
        en string pour la suite du projet"""
        for i in range(len(value)-1):
            instance_float.append(float(value[i]))
        """On ajoute ici la dernière valeur en conservant son type(string)"""
        instance_float.append(value[len(value)-1])
        data_float.append(instance_float)
    return data_float

"""Pour le jeu de données finalTest.txt on a pas la dernière colonne(la classe) alors on définit une nouvelle méthode
qui le prend en compte"""
def convert_string_float_final(data_string):
    data_float = []
    for value in data_string:
        instance_float = []
        for i in range(len(value)):
            instance_float.append(float(value[i]))
        data_float.append(instance_float)
    return data_float

data_float = convert_string_float(data_string)
#data_float_pre_test = convert_string_float(data_string_pre_test)
data_float_final_test = convert_string_float_final(data_string_final_test)

###  Couper en deux ensembles train/test afin d’entraîner notre modèle
- La division de notre ensemble de données est essentiel pour une évaluation des performances de prédiction. Dans la plupart des cas, dans notre cas nous allons diviser aléatoirement notre jeu de données en deux sous-ensembles: <code>train_set</code> et <code>test_set</code>.
- Pour le faire nous allons utiliser la méthode <code>train_test_set</code> qui prend en paramètre notre liste <code>data_float</code>, <code>split</code>(définie la taille du train_set et du test_set), <code>train_set</code> et <code>test_set</code> vide.

In [210]:
def train_test_set(data, split, train_set = [], test_set = []):
    for i in range(len(data)):
        """random.random() prends des valeurs aléatoires entre 0 et 1 pour séparer aléatoirement le jeu de données"""
        if random.random() < split:
            train_set.append(data[i])
        else:
            test_set.append(data[i])
            
train_set = []
test_set = []
"""70 % des données disponibles sont allouées à train_set. Les 30 % de données restantes 
sont réparties de manière égale dans le test_set."""
train_test_set(data_float, 0.70, train_set, test_set)

### Calculer la distance euclidienne
- L’algorithme repose sur la notion de « proximité » entre données. Cette proximité est mesurée à l’aide d’une distance.
- Nos données sont organisées dans <code>data_float</code> tel que chaque ligne correspond à un objet, et chaque colonne correspond à une valeur.
- La méthode <code>distance_euclidienne</code> calcule la distance entre deux lignes element1 et element2 de la façon suivante :

In [5]:
def distance_euclidienne(element1, element2):
    ecarts = [element1[j] - element2[j] for j in range(7)]
    #somme des carrés des écarts:
    sommeCarresEcarts = sum([ecart**2 for ecart in ecarts])
    return sqrt(sommeCarresEcarts)

### Algorithme k-nn
- Il est utilisé pour prédire la classe d’un nouvel échantillon en fonction de ses voisins les plus proches dans l’espace de caractéristiques.
- Pour le faire nous avons créer une méthode <code>knn</code> qui prend en paramètre un <code>element</code> et un <code>k</code> pour renvoyer la liste des k plus proches éléments correspondants.

In [6]:
def knn(element, k):

    # on crée la liste des distances entre element et les données du dataset train_set:
    distances = [distance_euclidienne(element, item) for item in data_float]

    # on ajoute dans une nouvelle list M des tuples avec les distances euclidiennes de element et un index:
    distance_index = [(distances[i], i) for i in range(len(distances))]
    
    # on trie distance_index suivant les distances, ordre croissant:
    distance_index.sort(key= lambda x: x[0])
    
    # On récupère uniquement les indices des k premiers éléments:
    top_k = [distance_index[i][1] for i in range(k)]
    
    # on renvoie la liste des éléments correspondants:
    return [data_float[k] for k in top_k]

### La classe la plus fréquente parmis les top k
- Ensuite après avoir trouver la liste des k premiers éléments on va retourner la classe la plus fréquente avec l'aide de la méthode <code>class_frequente</code>.

In [7]:
def classe_frequente(element, k):

    dico = {'0': 0, '1':0, '2':0, '3':0}
    # liste des voisins:
    voisins = knn(element, k)
    # mise à jour compteurs:
    for item in voisins:         
        dico[item[-1]] += 1  # element[-1] est  la classe (0 ou 1 ou 2 ou 3)

    # on cherche maintenant le maximum des valeurs du dico:
    effectifs_classe = [(clef, valeur) for clef, valeur in dico.items()] #liste de tuples: (classe, nbr de fois que classe apparaît)
    maxi = max(effectifs_classe, key= lambda x: x[1])
    return maxi[0]

### Précision de notre algorithme
- Cette méthode est utilisée pour trouver la précision de notre algorithme.

In [8]:
def get_accuracy(test_set, predictions):
    correct = 0
    for i in range(len(test_set)):
        if test_set[i][-1] is predictions[i]:
            correct += 1
    return (correct/float(len(test_set)))*100

### On exécute l'algorithm sur le dataset "finalTest.txt"
- On utilise le modèle afin de prédire la classe pour chaque exemple du fichier "finalTest.txt".Ensuite on créer un fichier de sortie "Sebastien MUGNIER_Sascha CAUCHON_TDC.txt".
- Ce fichier de sortie contiendra chaque prédiction (une prédiction par ligne :0,1,2 ou 3 suivie d’un retour à la ligne).

In [16]:
if __name__ == '__main__':        
    k = 1
    file_result = open('Sebastien MUGNIER_Sascha CAUCHON_TDC.txt', 'w')
    predictions = []
    for i in range(len(data_float_final_test)):
        nouvelElement = data_float_final_test[i]
        voisins = knn(nouvelElement, k)
        #predictions.append(classe_frequente(nouvelElement, k))
        file_result.write(f"{classe_frequente(nouvelElement, k)}\n")
    
    #accuracy = get_accuracy(data_float_pre_test, predictions)
    #file_result.write(f"Precision: {accuracy}")
    #print("Précision: ", accuracy)
    file_result.close()